In [21]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import time
from time import perf_counter
import glob
import json
import matplotlib.pyplot as plt
import plotly.express as px
import tweets_utils
import importlib

%autosave 3600

Autosaving every 3600 seconds


In [15]:
bots = pd.read_csv("C:/Users/Gianluca/Desktop/Supsi/Git/BachelorProject/csv/bot_score.csv")
bots

,id,user
0,1252818873578975232,matomecorona
1,1239490346691760128,Host24Space
2,1203462567861084160,naija_reports
3,1238121032067801088,shopfle
4,1257388917369667584,NbknewsC
...,...,...
609,1180444353577144320,edunewsid
610,91430932,PascalRtl
611,734775583842914304,news_naira
612,3231019148,JayaKatwang15


In [16]:
credibility_url = pd.read_csv("C:/Users/Gianluca/Desktop/Supsi/Git/BachelorProject/csv/domain_credibility.csv")
credibility_url

,Domain,Class
0,100percentfedup.com,low
1,100percentfedup.com,high
2,10news.one,low
3,12minutos.com,low
4,12news.com,high
...,...,...
820,childrenshealthdefense.org,low
821,"AdvancedMedicineConference.com,",low
822,aol.com,high
823,trendingpolitics.com,low


In [17]:
retweet_name = []
retweet_id = []
retweeted_name = []
retweeted_id = []
rt_link = []
tweet_name = []
tweet_id = []
link = []

result = []

total_len = 0
original_len = 0
retweet_len = 0
reply_len = 0
d_total_len = 0
d_original_len = 0
d_retweet_len = 0
d_reply_len = 0

cols_tweets = [u'user_screen_name',u'user_id', u'created_at', 
               u'urls', u"rt_created_at", u"rt_user_id", 
               u"rt_user_screen_name", u'in_reply_to_user_id', 
               u'in_reply_to_screen_name', u'in_reply_to_status_id']

files_tweets = glob.glob("F:/COVID-19-Tweets/*/tweets.csv")

In [24]:
importlib.reload(tweets_utils)
start_time = time.perf_counter()
j = 0
for filename in files_tweets:
    print(f"Reading file: {filename}")
    result.extend(tweets_utils.process_all_data(filename, cols_tweets, True, bots["user"]))
    lista = result[j]
    j = j + 1
    print(f"Mese numero {j}")
    for i in range(len(lista)):
        #original post
        tweet_name.extend(lista[i]["original_names"])
        tweet_id.extend(lista[i]["original_ids"])
        link.extend(lista[i]["links"])
#         #user who retweet
        retweet_name.extend(lista[i]["retweet_users"])
        retweet_id.extend(lista[i]["retweet_ids"])
#         #user retweeted
        retweeted_name.extend(lista[i]["retweeted_users"])
        retweeted_id.extend(lista[i]["retweeted_ids"])
        rt_link.extend(lista[i]["rt_link"])

        #len
        total_len = total_len + lista[i]["total_len"]
        original_len = original_len + lista[i]["original_len"]
        retweet_len = retweet_len + lista[i]["retweet_len"]
        reply_len = reply_len + lista[i]["reply_len"]
        d_total_len = d_total_len + lista[i]["d_total_len"]
        d_original_len = d_original_len + lista[i]["d_original_len"]
        d_retweet_len = d_retweet_len + lista[i]["d_retweet_len"]
        d_reply_len = d_reply_len + lista[i]["d_reply_len"]

stop_time = time.perf_counter()
print("Time: ",stop_time-start_time)

In [ ]:
tweets_utils.stats(total_len, original_len, retweet_len, reply_len)

In [ ]:
tweets_utils.print_pie_chart3("Total tweets stats","Original","Retweet", "Reply", original_len, retweet_len, reply_len)

In [ ]:
tweets_utils.stats(d_total_len, d_original_len, d_retweet_len, d_reply_len)

In [ ]:
tweets_utils.print_pie_chart3("Bot tweets stats","Original","Retweet", "Reply", d_original_len, d_retweet_len, d_reply_len)